# d4rl outils

In [1]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 21.8 MB/s eta 0:00:00


In [2]:
!pip install torch numpy matplotlib

In [3]:
!pip install just-d4rl

In [4]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 kB 4.6 MB/s eta 0:00:00


# Videos

In [18]:
import gymnasium as gym
import torch
import numpy as np
import os
import imageio
from stable_baselines3.common import policies
from just_d4rl import d4rl_offline_dataset
from stable_baselines3.common.torch_layers import FlattenExtractor, MlpExtractor
import torch.nn as nn


# Define UnconditionalPolicy
class UnconditionalPolicy(policies.ActorCriticPolicy):
    def __init__(
        self,
        observation_space,
        action_space,
        hidden_size=1024,
        depth=2,
        learning_rate=1e-3,
    ):
        net_arch = dict(
            pi=[hidden_size] * depth,
            vf=[64]
        )
        super().__init__(
            observation_space=observation_space,
            action_space=action_space,
            lr_schedule=lambda _: learning_rate,
            net_arch=net_arch,
            activation_fn=nn.ReLU,
            features_extractor_class=FlattenExtractor,
            optimizer_class=torch.optim.Adam,
        )

    def _build(self, lr_schedule):
        self.features_extractor = self.features_extractor_class(self.observation_space)
        self.features_dim = self.features_extractor.features_dim

        self.mlp_extractor = MlpExtractor(
            self.features_dim,
            net_arch=self.net_arch,
            activation_fn=self.activation_fn,
        )

        self.value_net = nn.Linear(self.mlp_extractor.latent_dim_vf, 1)

        latent_dim_pi = self.mlp_extractor.latent_dim_pi
        self.action_net, self.log_std = self.action_dist.proba_distribution_net(
            latent_dim=latent_dim_pi,
            log_std_init=self.log_std_init
        )


# Load the UnconditionalPolicy model
def load_model(model_class, model_path, observation_space, action_space):
    model = model_class(
        observation_space=observation_space,
        action_space=action_space,
        hidden_size=1024,
        depth=2
    )
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))
    model.eval()
    return model


# Set the OpenGL backend for headless rendering
os.environ["MUJOCO_GL"] = "egl"


# Function to save frames to video
def save_video_from_frames(frames, video_filename):
    writer = imageio.get_writer(video_filename, fps=60)
    for frame in frames:
        writer.append_data(frame)
    writer.close()
    print(f"Video saved at: {video_filename}")


# Evaluate the model and save video
def evaluate_model_and_save_video(model, env_name, video_filename, state_mean, state_std, num_evaluations=10, desired_return=12000 * 0.001):
    env = gym.make(env_name, render_mode="rgb_array")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    frames = []
    total_rewards = []

    for ep in range(num_evaluations):
        obs, _ = env.reset()
        total_reward = 0
        done = False
        desired_return_ep = desired_return  # Reset desired return for this episode

        while not done:
            # Normalize state
            state = (torch.tensor(obs, dtype=torch.float32).to(device) - state_mean) / state_std
            state = state.unsqueeze(0)

            # Add desired return to state
            desired_return_tensor = torch.tensor([[desired_return_ep]], dtype=torch.float32).to(device)
            augmented_state = torch.cat([state, desired_return_tensor], dim=1)

            # Predict action
            action = model._predict(augmented_state, deterministic=True).detach().cpu().numpy().flatten()
            action = np.clip(action, env.action_space.low, env.action_space.high)  # Clip actions

            # Step environment
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward

            # Update desired return
            desired_return_ep -= reward * 0.001
            frame = env.render()
            frames.append(frame)

        print(f"Sum of Reward for this episode: {total_reward}")
        total_rewards.append(total_reward)

    save_video_from_frames(frames, video_filename)
    avg_reward = np.mean(total_rewards)
    print(f"Average Reward over {num_evaluations} evaluations: {avg_reward}")
    return avg_reward


# Main function
def main():
    dataset_name = "halfcheetah-medium-expert-v2"
    model_path = f"trained_mle_model_{dataset_name}.pth"
    video_filename = f"./{dataset_name}_evaluation.mp4"

    env_name = dataset_name.split('-')[0].replace('halfcheetah', 'HalfCheetah') + '-v4'
    env = gym.make(env_name, render_mode="rgb_array")

    action_space = env.action_space
    augmented_obs_space = gym.spaces.Box(
        low=np.concatenate([env.observation_space.low, [-np.inf]]),
        high=np.concatenate([env.observation_space.high, [np.inf]]),
        dtype=np.float32
    )

    # Load the model
    model = load_model(UnconditionalPolicy, model_path, augmented_obs_space, action_space)

    # Load dataset statistics for normalization
    dataset = d4rl_offline_dataset(dataset_name)
    states = dataset['observations']
    state_mean = torch.tensor(states.mean(axis=0), dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')
    state_std = torch.tensor(states.std(axis=0) + 1e-6, dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')

    # Evaluate the model and save video
    avg_reward = evaluate_model_and_save_video(
        model,
        env_name,
        video_filename,
        state_mean,
        state_std,
        num_evaluations=10,
        desired_return=12000 * 0.001
    )

    print(f"Final Average Reward for {dataset_name}: {avg_reward}")


if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:517: DeprecationWarning: WARN: The environment HalfCheetah-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(
<ipython-input-18-7fa3f4aa3cb0>:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don

Dataset loaded and saved at: /root/.d4rl/datasets/halfcheetah_medium_expert-v2.hdf5
Sum of Reward for this episode: 11624.95245774274
Sum of Reward for this episode: 11218.757056662911
Sum of Reward for this episode: 11279.934292811216
Sum of Reward for this episode: 11289.573013770796
Sum of Reward for this episode: 11457.571741994941
Sum of Reward for this episode: 11162.789090940802
Sum of Reward for this episode: 11290.620105226268
Sum of Reward for this episode: 11247.596560491667
Sum of Reward for this episode: 11258.79190017032
Sum of Reward for this episode: 11190.02640530622
Video saved at: ./halfcheetah-medium-expert-v2_evaluation.mp4
Average Reward over 10 evaluations: 11302.061262511788
Final Average Reward for halfcheetah-medium-expert-v2: 11302.061262511788


In [19]:
import gymnasium as gym
import torch
import numpy as np
import os
import imageio
from stable_baselines3.common import policies
from just_d4rl import d4rl_offline_dataset
from stable_baselines3.common.torch_layers import FlattenExtractor, MlpExtractor
import torch.nn as nn


# Define UnconditionalPolicy
class UnconditionalPolicy(policies.ActorCriticPolicy):
    def __init__(
        self,
        observation_space,
        action_space,
        hidden_size=1024,
        depth=2,
        learning_rate=1e-3,
    ):
        net_arch = dict(
            pi=[hidden_size] * depth,
            vf=[64]
        )
        super().__init__(
            observation_space=observation_space,
            action_space=action_space,
            lr_schedule=lambda _: learning_rate,
            net_arch=net_arch,
            activation_fn=nn.ReLU,
            features_extractor_class=FlattenExtractor,
            optimizer_class=torch.optim.Adam,
        )

    def _build(self, lr_schedule):
        self.features_extractor = self.features_extractor_class(self.observation_space)
        self.features_dim = self.features_extractor.features_dim

        self.mlp_extractor = MlpExtractor(
            self.features_dim,
            net_arch=self.net_arch,
            activation_fn=self.activation_fn,
        )

        self.value_net = nn.Linear(self.mlp_extractor.latent_dim_vf, 1)

        latent_dim_pi = self.mlp_extractor.latent_dim_pi
        self.action_net, self.log_std = self.action_dist.proba_distribution_net(
            latent_dim=latent_dim_pi,
            log_std_init=self.log_std_init
        )


# Load the UnconditionalPolicy model
def load_model(model_class, model_path, observation_space, action_space):
    model = model_class(
        observation_space=observation_space,
        action_space=action_space,
        hidden_size=1024,
        depth=2
    )
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))
    model.eval()
    return model


# Set the OpenGL backend for headless rendering
os.environ["MUJOCO_GL"] = "egl"


# Function to save frames to video
def save_video_from_frames(frames, video_filename):
    writer = imageio.get_writer(video_filename, fps=60)
    for frame in frames:
        writer.append_data(frame)
    writer.close()
    print(f"Video saved at: {video_filename}")


# Evaluate the model and save video
def evaluate_model_and_save_video(model, env_name, video_filename, state_mean, state_std, num_evaluations=10, desired_return=3600 * 0.001):
    env = gym.make(env_name, render_mode="rgb_array")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    frames = []
    total_rewards = []

    for ep in range(num_evaluations):
        obs, _ = env.reset()
        total_reward = 0
        done = False
        desired_return_ep = desired_return  # Reset desired return for this episode

        while not done:
            # Normalize state
            state = (torch.tensor(obs, dtype=torch.float32).to(device) - state_mean) / state_std
            state = state.unsqueeze(0)

            # Add desired return to state
            desired_return_tensor = torch.tensor([[desired_return_ep]], dtype=torch.float32).to(device)
            augmented_state = torch.cat([state, desired_return_tensor], dim=1)

            # Predict action
            action = model._predict(augmented_state, deterministic=True).detach().cpu().numpy().flatten()
            action = np.clip(action, env.action_space.low, env.action_space.high)  # Clip actions

            # Step environment
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward

            # Update desired return
            desired_return_ep -= reward * 0.001
            frame = env.render()
            frames.append(frame)

        print(f"Sum of Reward for this episode: {total_reward}")
        total_rewards.append(total_reward)

    save_video_from_frames(frames, video_filename)
    avg_reward = np.mean(total_rewards)
    print(f"Average Reward over {num_evaluations} evaluations: {avg_reward}")
    return avg_reward


# Main function
def main():
    dataset_name = "hopper-medium-expert-v2"
    model_path = f"trained_mle_model_{dataset_name}.pth"
    video_filename = f"./{dataset_name}_evaluation.mp4"

    env_name = dataset_name.split('-')[0].replace('hopper', 'Hopper') + '-v4'
    env = gym.make(env_name, render_mode="rgb_array")

    action_space = env.action_space
    augmented_obs_space = gym.spaces.Box(
        low=np.concatenate([env.observation_space.low, [-np.inf]]),
        high=np.concatenate([env.observation_space.high, [np.inf]]),
        dtype=np.float32
    )

    # Load the model
    model = load_model(UnconditionalPolicy, model_path, augmented_obs_space, action_space)

    # Load dataset statistics for normalization
    dataset = d4rl_offline_dataset(dataset_name)
    states = dataset['observations']
    state_mean = torch.tensor(states.mean(axis=0), dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')
    state_std = torch.tensor(states.std(axis=0) + 1e-6, dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')

    # Evaluate the model and save video
    avg_reward = evaluate_model_and_save_video(
        model,
        env_name,
        video_filename,
        state_mean,
        state_std,
        num_evaluations=10,
        desired_return=3600 * 0.001
    )

    print(f"Final Average Reward for {dataset_name}: {avg_reward}")


if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:517: DeprecationWarning: WARN: The environment Hopper-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(
<ipython-input-19-f5eaa4d3a4ae>:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't ha

Dataset loaded and saved at: /root/.d4rl/datasets/hopper_medium_expert-v2.hdf5
Sum of Reward for this episode: 3384.514099871218
Sum of Reward for this episode: 2494.3416450073314
Sum of Reward for this episode: 2579.4993052006303
Sum of Reward for this episode: 3566.669651624607
Sum of Reward for this episode: 3632.1642348205987
Sum of Reward for this episode: 3368.264849530412
Sum of Reward for this episode: 1574.8869895142625
Sum of Reward for this episode: 3567.374677517645
Sum of Reward for this episode: 3592.376119709841
Sum of Reward for this episode: 3508.5736931077454
Video saved at: ./hopper-medium-expert-v2_evaluation.mp4
Average Reward over 10 evaluations: 3126.866526590429
Final Average Reward for hopper-medium-expert-v2: 3126.866526590429


In [20]:
import gymnasium as gym
import torch
import numpy as np
import os
import imageio
from stable_baselines3.common import policies
from just_d4rl import d4rl_offline_dataset
from stable_baselines3.common.torch_layers import FlattenExtractor, MlpExtractor
import torch.nn as nn


# Define UnconditionalPolicy
class UnconditionalPolicy(policies.ActorCriticPolicy):
    def __init__(
        self,
        observation_space,
        action_space,
        hidden_size=1024,
        depth=2,
        learning_rate=1e-3,
    ):
        net_arch = dict(
            pi=[hidden_size] * depth,
            vf=[64]
        )
        super().__init__(
            observation_space=observation_space,
            action_space=action_space,
            lr_schedule=lambda _: learning_rate,
            net_arch=net_arch,
            activation_fn=nn.ReLU,
            features_extractor_class=FlattenExtractor,
            optimizer_class=torch.optim.Adam,
        )

    def _build(self, lr_schedule):
        self.features_extractor = self.features_extractor_class(self.observation_space)
        self.features_dim = self.features_extractor.features_dim

        self.mlp_extractor = MlpExtractor(
            self.features_dim,
            net_arch=self.net_arch,
            activation_fn=self.activation_fn,
        )

        self.value_net = nn.Linear(self.mlp_extractor.latent_dim_vf, 1)

        latent_dim_pi = self.mlp_extractor.latent_dim_pi
        self.action_net, self.log_std = self.action_dist.proba_distribution_net(
            latent_dim=latent_dim_pi,
            log_std_init=self.log_std_init
        )


# Load the UnconditionalPolicy model
def load_model(model_class, model_path, observation_space, action_space):
    model = model_class(
        observation_space=observation_space,
        action_space=action_space,
        hidden_size=1024,
        depth=2
    )
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))
    model.eval()
    return model


# Set the OpenGL backend for headless rendering
os.environ["MUJOCO_GL"] = "egl"


# Function to save frames to video
def save_video_from_frames(frames, video_filename):
    writer = imageio.get_writer(video_filename, fps=60)
    for frame in frames:
        writer.append_data(frame)
    writer.close()
    print(f"Video saved at: {video_filename}")


# Evaluate the model and save video
def evaluate_model_and_save_video(model, env_name, video_filename, state_mean, state_std, num_evaluations=10, desired_return=5000 * 0.001):
    env = gym.make(env_name, render_mode="rgb_array")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    frames = []
    total_rewards = []

    for ep in range(num_evaluations):
        obs, _ = env.reset()
        total_reward = 0
        done = False
        desired_return_ep = desired_return  # Reset desired return for this episode

        while not done:
            # Normalize state
            state = (torch.tensor(obs, dtype=torch.float32).to(device) - state_mean) / state_std
            state = state.unsqueeze(0)

            # Add desired return to state
            desired_return_tensor = torch.tensor([[desired_return_ep]], dtype=torch.float32).to(device)
            augmented_state = torch.cat([state, desired_return_tensor], dim=1)

            # Predict action
            action = model._predict(augmented_state, deterministic=True).detach().cpu().numpy().flatten()
            action = np.clip(action, env.action_space.low, env.action_space.high)  # Clip actions

            # Step environment
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward

            # Update desired return
            desired_return_ep -= reward * 0.001
            frame = env.render()
            frames.append(frame)

        print(f"Sum of Reward for this episode: {total_reward}")
        total_rewards.append(total_reward)

    save_video_from_frames(frames, video_filename)
    avg_reward = np.mean(total_rewards)
    print(f"Average Reward over {num_evaluations} evaluations: {avg_reward}")
    return avg_reward


# Main function
def main():
    dataset_name = "walker2d-medium-expert-v2"
    model_path = f"trained_mle_model_{dataset_name}.pth"
    video_filename = f"./{dataset_name}_evaluation.mp4"

    env_name = dataset_name.split('-')[0].replace('walker2d', 'Walker2d') + '-v4'
    env = gym.make(env_name, render_mode="rgb_array")

    action_space = env.action_space
    augmented_obs_space = gym.spaces.Box(
        low=np.concatenate([env.observation_space.low, [-np.inf]]),
        high=np.concatenate([env.observation_space.high, [np.inf]]),
        dtype=np.float32
    )

    # Load the model
    model = load_model(UnconditionalPolicy, model_path, augmented_obs_space, action_space)

    # Load dataset statistics for normalization
    dataset = d4rl_offline_dataset(dataset_name)
    states = dataset['observations']
    state_mean = torch.tensor(states.mean(axis=0), dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')
    state_std = torch.tensor(states.std(axis=0) + 1e-6, dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')

    # Evaluate the model and save video
    avg_reward = evaluate_model_and_save_video(
        model,
        env_name,
        video_filename,
        state_mean,
        state_std,
        num_evaluations=10,
        desired_return=5000 * 0.001
    )

    print(f"Final Average Reward for {dataset_name}: {avg_reward}")


if __name__ == "__main__":
    main()

<ipython-input-20-f4ba6405f2d9>:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))
load dataf

Dataset loaded and saved at: /root/.d4rl/datasets/walker2d_medium_expert-v2.hdf5
Sum of Reward for this episode: 4904.8224526156
Sum of Reward for this episode: 4870.28563317747
Sum of Reward for this episode: 4868.763126591738
Sum of Reward for this episode: 4888.087381968669
Sum of Reward for this episode: 4883.4723337021205
Sum of Reward for this episode: 4905.227923898287
Sum of Reward for this episode: 4908.204982869992
Sum of Reward for this episode: 4899.835439343304
Sum of Reward for this episode: 4883.56162887681
Sum of Reward for this episode: 4894.967023848706
Video saved at: ./walker2d-medium-expert-v2_evaluation.mp4
Average Reward over 10 evaluations: 4890.72279268927
Final Average Reward for walker2d-medium-expert-v2: 4890.72279268927
